In [45]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions





In [68]:
model = tf.keras.applications.MobileNetV2()


In [46]:
test_dir = '/content/drive/MyDrive/strabismus cnn model/testing'
train_dir = '/content/drive/MyDrive/strabismus cnn model/training '

In [47]:
img_width, img_height = 160, 160
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 279 images belonging to 2 classes.
Found 289 images belonging to 2 classes.


In [48]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(img_width, img_height, 3),
    include_top=False,
    weights='imagenet'
)

In [49]:
base_model.trainable = False

In [50]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(train_generator.num_classes, activation='sigmoid')
layer = tf.keras.layers.Dense(128, activation='relu')
layer2 = tf.keras.layers.Dense(64, activation='relu')
layer3 = tf.keras.layers.Dense(32, activation='relu')
layer4 = tf.keras.layers.Dense(16, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
dropout_layer2 = tf.keras.layers.Dropout(0.25)
dropout_layer3 = tf.keras.layers.Dropout(0.125)

In [67]:
model = tf.keras.Sequential([
    base_model,
    global_average_layer,
    layer,
    dropout_layer3,

    prediction_layer
])

In [57]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [58]:

history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=8,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)



Epoch 1/8
9/9 [==============================] - 24s 2s/step - loss: 0.1254 - accuracy: 0.9498 - val_loss: 0.5737 - val_accuracy: 0.8097
Epoch 2/8
9/9 [==============================] - 16s 2s/step - loss: 0.1022 - accuracy: 0.9677 - val_loss: 0.6563 - val_accuracy: 0.7855
Epoch 3/8
9/9 [==============================] - 15s 2s/step - loss: 0.0838 - accuracy: 0.9749 - val_loss: 0.5933 - val_accuracy: 0.8131
Epoch 4/8
9/9 [==============================] - 20s 2s/step - loss: 0.0832 - accuracy: 0.9713 - val_loss: 0.6903 - val_accuracy: 0.7958
Epoch 5/8
9/9 [==============================] - 17s 2s/step - loss: 0.0735 - accuracy: 0.9785 - val_loss: 0.6206 - val_accuracy: 0.8062
Epoch 6/8
9/9 [==============================] - 15s 2s/step - loss: 0.0648 - accuracy: 0.9785 - val_loss: 0.6457 - val_accuracy: 0.8028
Epoch 7/8
9/9 [==============================] - 18s 2s/step - loss: 0.0788 - accuracy: 0.9713 - val_loss: 0.6081 - val_accuracy: 0.8166
Epoch 8/8
9/9 [==========================

In [62]:

image_path = input('Enter the path of the image: ')
img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)

predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

class_names = train_generator.class_indices
class_name = list(class_names.keys())[list(class_names.values()).index(predicted_class)]

print(f'The image belongs to class: {class_name}')


Enter the path of the image: /content/drive/MyDrive/strabismus cnn model/testing/normal eyes/Advantages-of-having-Two-Eyes-1.png
1/1 [==============================] - 0s 47ms/step
The image belongs to class: normal eyes
